Step 1: Install Required Libraries
- !pip install psycopg2-binary
- !pip install sentence-transformers
- !pip install openai  # Only if you plan to use OpenAI's API

Step 2: Connect to PostgreSQL and Retrieve Data

In [ ]:
import psycopg2
import pandas as pd

# Database connection details
db_config = {
    'host': 'localhost',
    'database': 'recipe_db',
    'user': 'postgres',
    'password': '',
    'port': '5433'  
}

# Establish connection to PostgreSQL
conn = psycopg2.connect(**db_config)

# Retrieve recipes and ingredients data
query = """
SELECT r.name, r.description, n.calories, n.total_fat, n.protein, n.carbs, 
       i.ingredient_text, d.instruction_text
FROM recipes r
JOIN nutrition_facts n ON r.recipe_id = n.recipe_id
JOIN ingredients i ON r.recipe_id = i.recipe_id
JOIN directions d ON r.recipe_id = d.recipe_id;
"""

# Load data into a DataFrame
recipes_df = pd.read_sql(query, conn)
conn.close()

# Preview the retrieved data
print(recipes_df.head())




C:\Users\ancha\AppData\Local\Temp\ipykernel_38076\1466140673.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recipes_df = pd.read_sql(query, conn)


                                    name  \
0  Peanut Butter Banana Protein Smoothie   
1  Peanut Butter Banana Protein Smoothie   
2  Peanut Butter Banana Protein Smoothie   
3  Peanut Butter Banana Protein Smoothie   
4  Peanut Butter Banana Protein Smoothie   

                                         description  calories  total_fat  \
0  A nutritious and satisfying beverage for break...       482       8.34   
1  A nutritious and satisfying beverage for break...       482       8.34   
2  A nutritious and satisfying beverage for break...       482       8.34   
3  A nutritious and satisfying beverage for break...       482       8.34   
4  A nutritious and satisfying beverage for break...       482       8.34   

   protein  carbs              ingredient_text  \
0    28.57  77.97                        Honey   
1    28.57  77.97                        Honey   
2    28.57  77.97                        Honey   
3    28.57  77.97  Oikos Triple Zero - Vanilla   
4    28.57  77.97  Oik


- The code successfully retrieved data from the PostgreSQL database, showing a sample output with five rows. Each row includes:

    - name: Name of the recipe
    - description: Description of the recipe
    - calories, total_fat, protein, carbs: Nutritional information
    - ingredient_text: Ingredients for the recipe
    - instruction_text: Step-by-step instructions
- The warning about using SQLAlchemy can be ignored here, as the query still succeeded.

Step 3: Data Preprocessing
- Combine the recipe description, ingredients, and directions into a single text field for each recipe.

In [21]:
# Combine fields into a single text field for each recipe
recipes_df['text'] = recipes_df['description'] + " " + recipes_df['ingredient_text'] + " " + recipes_df['instruction_text']

# Group by recipe name to aggregate all ingredients and instructions per recipe
recipes_texts = recipes_df.groupby('name')['text'].apply(' '.join).reset_index()


Step 4: Generate Embeddings
- Use a pre-trained model like Sentence Transformers to generate embeddings for each recipe.

In [22]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models if preferred

# Generate embeddings for each recipe text
recipes_texts['embedding'] = recipes_texts['text'].apply(lambda x: model.encode(x))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\ancha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ancha\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


- The model, sentence-transformers/all-MiniLM-L6-v2, was successfully loaded, but with a warning related to caching limitations on Windows. This warning indicates that caching files might use more disk space if symbolic links are not supported, which is generally a minor concern unless disk space is limited.

Step 5 (Optional): Save Embeddings to Database
- If you want to save these embeddings back to the database for faster retrieval, you'll need to serialize them. Here’s an example using PostgreSQL.

In [23]:
import numpy as np
from sqlalchemy import create_engine

# Establish connection using SQLAlchemy for easier writing back
engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config.get('port', 5433)}/{db_config['database']}")

# Serialize embeddings as binary data
recipes_texts['embedding'] = recipes_texts['embedding'].apply(lambda x: np.array(x).tobytes())

# Save to a new table (or update an existing one)
recipes_texts[['name', 'embedding']].to_sql('recipe_embeddings', engine, if_exists='replace', index=False)


566

The output 566 indicates that embeddings were generated for 566 recipes in the dataset.

Step 6: Build a Simple Recommendation Function
- Calculate similarity scores between the user’s input and each recipe based on the embeddings.

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_recipes(user_input, recipes_texts, model, top_n=5):
    # Generate embedding for user input
    user_embedding = model.encode(user_input)

    # Calculate similarity scores
    recipes_texts['similarity'] = recipes_texts['embedding'].apply(
        lambda x: cosine_similarity([user_embedding], [np.frombuffer(x, dtype=np.float32)]).flatten()[0]
    )

    # Sort by similarity and return top N recommendations
    recommendations = recipes_texts.sort_values(by='similarity', ascending=False).head(top_n)
    return recommendations[['name', 'similarity']]

# Example usage
user_input = "high protein low carb breakfast"
recommendations = recommend_recipes(user_input, recipes_texts, model)
print(recommendations)


                                           name  similarity
1626  Ham Cheddar and Jalapeno Crustless Quiche    0.668146
1132                       Cream Cheese Biscuit    0.627648
3343                   Vanilla Protein Pancakes    0.613065
299                 Banana Stuffed French Toast    0.611616
2982         Strawberry Banana Protein Smoothie    0.598489



- The code provided recommendations based on the similarity of the recipes to a given user input ("high protein low carb breakfast"). The top 5 recommended recipes are:

    - Ham Cheddar and Jalapeno Crustless Quiche - Similarity: 0.668146
    - Cream Cheese Biscuit - Similarity: 0.627648
    - Vanilla Protein Pancakes - Similarity: 0.613065
    - Banana Stuffed French Toast - Similarity: 0.611616
    - Strawberry Banana Protein Smoothie - Similarity: 0.598489
- Each recipe in the recommendation list has a similarity score, indicating how closely it matches the user’s input based on the embeddings generated.

Explanation of Each Step
- Data Retrieval: We connect to the database, query the necessary fields, and load the data into a Pandas DataFrame.
- Preprocessing: Combine descriptions, ingredients, and directions for each recipe to create a unified text field, which improves the quality of embeddings.
- Generate Embeddings: Use sentence-transformers to encode the text data into numerical vectors. These embeddings allow for similarity calculations.
Save Embeddings (Optional): Save embeddings back to the database, serialized as binary data. This is optional but useful for faster future access.
- Recommendation Function: Calculate cosine similarity between the user’s input embedding and each recipe's embedding to find the most relevant recipes.
Notes
- Embeddings Storage: If you plan to use embeddings frequently, storing them back in the database can be efficient, though it requires serialization.
- Model Choice: You can experiment with different Sentence Transformers models or use OpenAI's API if preferred (note that OpenAI usage incurs costs).

In [ ]:
from sentence_transformers import SentenceTransformer
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Database configuration
db_config = {
    'host': 'localhost',
    'port': '5433',
    'database': 'recipe_db',
    'user': 'postgres',
    'password': ''
}

# Establish connection using SQLAlchemy for easier database operations
engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# Step 1: Retrieve recipes and ingredients
query = """
SELECT r.recipe_id, r.description, i.ingredient_text
FROM recipes r
JOIN ingredients i ON r.recipe_id = i.recipe_id;
"""
recipes_df = pd.read_sql(query, engine)

# Combine description and ingredient_text for each recipe
recipes_df['combined_text'] = recipes_df['description'] + " " + recipes_df['ingredient_text']

# Group by recipe_id to aggregate all ingredient_text entries per recipe
recipes_texts = recipes_df.groupby('recipe_id')['combined_text'].apply(' '.join).reset_index()

# Step 2: Generate embeddings for each recipe
recipes_texts['recipe_vector'] = recipes_texts['combined_text'].apply(lambda x: model.encode(x))

# Step 3: Save the embeddings back to the database
with engine.connect() as conn:
    for idx, row in recipes_texts.iterrows():
        # Convert the embedding to a format suitable for storage (e.g., as a string)
        vector_str = np.array2string(row['recipe_vector'], separator=',')
        conn.execute(
            f"""
            UPDATE recipes
            SET recipe_vector = %s
            WHERE recipe_id = %s
            """,
            (vector_str, row['recipe_id'])
        )

print("Embeddings have been successfully stored in the database.")



Embeddings have been successfully stored in the database.
